In [ ]:
# Copyright © 2021 LVCS. All Rights Reserved
import numpy as np
import pandas as pd
import copy
import random
from scipy.special import comb


In [ ]:
# 导入数据，初始化
cluster_result_raw = np.loadtxt("in.csv", delimiter=",")
total = np.sum(cluster_result_raw != 0)
cluster_done = np.zeros(total + 1, dtype=int)


In [ ]:
# 整理聚类
for i in range(len(cluster_result_raw)):
    for j in range(len(cluster_result_raw[0][:])):
        if cluster_result_raw[i][j] == 0:
            break
        else:
            cluster_done[int(cluster_result_raw[i, j])] = str(i + 1)

# 针对缺少元素的处理
# a = np.where(cluster_done == 0)
# cluster_done[a[0][1]] = (len(cluster_result_raw))

# 删除多余项
cluster_done = np.delete(cluster_done, cluster_done == 0)

# 转换为df类型，方便输出
cluster_done = pd.DataFrame(cluster_done)
cluster_done.columns = ["cluster"]


In [ ]:
# 导入原始表格
origin = pd.read_csv("1.csv")
origin.insert(1, "cluster", cluster_done)
origin = origin.sort_values(by=["cluster", "symbol"])
origin.insert(2, "num", np.linspace(1, total, endpoint=True, num=total, dtype=int))
x = copy.deepcopy(origin)


In [ ]:
# 删除多余数据
del x["cluster"]
del x["symbol"]
del x["num"]
x = x.values


In [ ]:
# 计算总表格皮尔森相关系数
pc = np.corrcoef(x)
pc = np.abs(pc)


In [ ]:
# 计算DNB
for ii in range(1, origin.max().cluster + 1):
    # 计算当前聚类起始、结束位置
    start = origin[(origin["cluster"] == ii)].min().num
    end = origin[(origin["cluster"] == ii)].max().num
    pccin = np.triu(pc[start - 1 : end, start - 1 : end])
    pccin_ave = (np.sum(pccin) - (end - start + 1)) / comb(end - start + 1, 2)
    sdin = 0
    for i in range(start - 1, end):
        sdin = sdin + np.std(x[i, :], ddof=1)
        sdin_ave = sdin / (end - start + 1)
    i = 1
    pccout = 0
    # PCCout元素随机选取
    times = int(len(pc) / 5)
    while i <= times:
        rand = random.randint(0, len(x) - 1)
        if rand >= start and rand <= end:
            continue
        else:
            pccout = pccout + np.sum(pc[rand, start - 1 : end]) / (end - start + 1)
            i = i + 1
    pccout_ave = pccout / times
    dnb = pccin_ave * sdin_ave / pccout_ave
    print(dnb)
